In [ ]:
import os

import pandas as pd
from geopandas import gpd

from covidcaremap.data import published_data_path, processed_data_path
from covidcaremap.geo import sum_per_region

In [ ]:
ccm_facility_data_gpd = gpd.read_file(published_data_path(
    'us_healthcare_capacity-facility-CovidCareMap.geojson'))

In [ ]:
# Read in county-level data
county_gdf = gpd.read_file(processed_data_path('us_counties_with_pop.geojson'))

# Filter by counties of interest
counties_of_interest = ['Alamedac County',
                        'Marin County', 
                        'Sonoma County', 
                        'Napa County', 
                        'Solano County', 
                        'Contra Costa County',
                        'Santa Cruz County']

filtered_county_gdf = county_gdf
filtered_county_gdf['Region Name'] = filtered_county_gdf['County Name'] + ' County'
filtered_county_gdf = filtered_county_gdf[filtered_county_gdf['Region Name'].isin(counties_of_interest)]
filtered_county_gdf = filtered_county_gdf[['Region Name', 'Population', 'geometry']]
filtered_county_gdf

In [ ]:
# Get san francisco GeoJSON        
sf_neighborhoods_gdf = gpd.read_file('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/san-francisco.geojson')


In [ ]:
sf_neighborhoods_gdf['region_id'] = 1
sf_gdf = sf_neighborhoods_gdf.dissolve(by='region_id')[['geometry']]

# Source: 2017 Census data
sf_gdf['Region Name'] = 'San Francisco'
sf_gdf['Population'] = 884363

In [ ]:
sf_gdf

In [ ]:
counties_and_sf_gdf = pd.concat([filtered_county_gdf,sf_gdf])

In [ ]:
counties_and_sf_gdf

In [ ]:
result_gdf = sum_per_region(ccm_facility_data_gpd,
                   counties_and_sf_gdf,
                   groupby_columns='Region Name',
                   region_id_column='Region Name',
                   population_columns={'People': 'Population'})
result_df = result_gdf[['Region Name',
                        'Staffed ICU Beds',
                        'Staffed All Beds',
                        'Licensed All Beds',
                        'All Bed Occupancy Rate',
                        'ICU Bed Occupancy Rate',
                        'Staffed ICU Beds [Per 1000 People]',
                        'Staffed All Beds [Per 1000 People]',
                        'Licensed All Beds [Per 1000 People]']]
result_df

In [ ]:
result_df.to_csv('sf-vs-bay-area.csv')